<a href="https://colab.research.google.com/github/AnanyaSourav/Minor-Skin-Cancer-Project/blob/main/RinColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to use R in Google Colab
### There are two ways to run R in Colab
- The first way is to use the rpy2 package in the Python runtime. This method allows you to execute R and Python syntax together.
- The second way is to actually start the notebook in the R runtime.
### How to use R and Python together in Colab
- Open google browser.
- Create a new notebook: https://colab.research.google.com/#create=true.
- Run rmagic by executing this command `%load_ext rpy2.ipython`.
- After that, every time you want to use R, add `%%R` in the beginning of each cell.


In [1]:
# Install necessary packages
install.packages(c("data.table", "ggplot2", "igraph", "gtools", "cluster", "factoextra"))

# Load libraries
library(data.table)
library(ggplot2)
library(igraph)
library(gtools)
library(cluster)
library(factoextra)

# Define input/output paths
input_folder <- "/content/sample_data/merged"
output_folder <- "/content/sample_data/Analysis_Results"
dir.create(output_folder, showWarnings = FALSE)

# Load all CSV files
file_list <- list.files(input_folder, pattern = "*.csv", full.names = TRUE)

# Initialize empty list to store data
all_data <- list()

# Read and clean data
for (file in file_list) {
    df <- fread(file)

    # Ensure correct column names
    if (!all(c("HGNC_Symbol", "log2R_CNA", "Z_Score") %in% colnames(df))) {
        next  # Skip files with incorrect format
    }

    # Remove duplicates
    df <- unique(df)

    # Handle missing values (impute with median)
    df[, log2R_CNA := ifelse(is.na(log2R_CNA), median(log2R_CNA, na.rm = TRUE), log2R_CNA)]
    df[, Z_Score := ifelse(is.na(Z_Score), median(Z_Score, na.rm = TRUE), Z_Score)]

    # Normalize values
    df[, log2R_CNA := scale(log2R_CNA)]
    df[, Z_Score := scale(Z_Score)]

    # Store in list
    patient_id <- tools::file_path_sans_ext(basename(file))
    all_data[[patient_id]] <- df
}

# Convert list to combined data table
combined_data <- rbindlist(all_data, idcol = "Patient_ID")

# Compute variance & standard deviation
variance_stats <- combined_data[, .(
    Mean_CNA = mean(log2R_CNA, na.rm = TRUE),
    SD_CNA = sd(log2R_CNA, na.rm = TRUE),
    Mean_Exp = mean(Z_Score, na.rm = TRUE),
    SD_Exp = sd(Z_Score, na.rm = TRUE)
), by = HGNC_Symbol]

# Save variance statistics
fwrite(variance_stats, file.path(output_folder, "variance_statistics.csv"))

# Visualize variance distribution
png(file.path(output_folder, "variance_distribution.png"), width = 800, height = 600)
ggplot(variance_stats, aes(x = SD_CNA, y = SD_Exp)) +
    geom_point(alpha = 0.6) +
    labs(title = "Variance Distribution: CNA vs Expression",
         x = "CNA Standard Deviation",
         y = "Expression Standard Deviation") +
    theme_minimal()
dev.off()

# Network Analysis: Build gene interaction graph
graph_data <- combined_data[, .(
    Correlation = cor(log2R_CNA, Z_Score, use = "complete.obs")
), by = HGNC_Symbol]

# Filter significant interactions
graph_data <- graph_data[abs(Correlation) > 0.5]

# Create graph
gene_graph <- graph_from_data_frame(graph_data, directed = FALSE)
png(file.path(output_folder, "gene_network.png"), width = 800, height = 600)
plot(gene_graph, vertex.size = 5, vertex.label.cex = 0.7, main = "Gene Interaction Network")
dev.off()

# Clustering patients based on CNA-Expression profiles
patient_matrix <- dcast(combined_data, Patient_ID ~ HGNC_Symbol, value.var = "log2R_CNA")
patient_matrix <- as.matrix(patient_matrix[, -1, with = FALSE])  # Remove Patient_ID column

# Check for NA values
if (any(is.na(patient_matrix))) {
    patient_matrix[is.na(patient_matrix)] <- 0
}

# Perform hierarchical clustering
dist_matrix <- dist(patient_matrix, method = "euclidean")
hc <- hclust(dist_matrix, method = "ward.D2")

# Save dendrogram
png(file.path(output_folder, "patient_clustering.png"), width = 800, height = 600)
plot(hc, main = "Hierarchical Clustering of Patients", xlab = "", sub = "")
dev.off()

# Save final processed dataset
fwrite(combined_data, file.path(output_folder, "final_processed_data.csv"))


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘Deriv’, ‘microbenchmark’, ‘Rdpack’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘lazyeval’, ‘carData’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘crosstalk’, ‘estimability’, ‘numDeriv’, ‘mvtnorm’, ‘corrplot’, ‘viridis’, ‘car’, ‘DT’, ‘ellipse’, ‘emmeans’, ‘flashClust’, ‘leaps’, ‘multcompView’, ‘scatterplot3d’, ‘ggsci’, ‘cowplot’, ‘ggsignif’, ‘gridExtra’, ‘polynom’, ‘rstatix’, ‘plyr’, ‘abind’, ‘dendextend’, ‘FactoMineR’, ‘ggpubr’, ‘reshape2’, ‘ggrepel’



Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘gtools’


The following object is masked from ‘package:igraph’:

    permute


Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



pdf 
  2

pdf 
  2

pdf 
  2

In [1]:
# Install required libraries (if not installed)
if (!require("ggplot2")) install.packages("ggplot2", dependencies=TRUE)
if (!require("igraph")) install.packages("igraph", dependencies=TRUE)
if (!require("dplyr")) install.packages("dplyr", dependencies=TRUE)
if (!require("tidyr")) install.packages("tidyr", dependencies=TRUE)
if (!require("pheatmap")) install.packages("pheatmap", dependencies=TRUE)

# Load libraries
library(ggplot2)
library(igraph)
library(dplyr)
library(tidyr)
library(pheatmap)

# Set file paths for Google Colab
input_file <- "/content/sample_data/Analysis_Results/variance_statistics.csv"
output_dir <- "/content/sample_data/Analysis_Results/"

# Create output directory if it doesn't exist
if (!dir.exists(output_dir)) dir.create(output_dir, recursive = TRUE)

# Read the data
data <- read.csv(input_file, stringsAsFactors = FALSE)

# Check and handle missing values
data <- na.omit(data)  # Remove rows with missing values

# Remove duplicates based on HGNC_Symbol
data <- data[!duplicated(data$HGNC_Symbol), ]

# Remove extreme outliers using IQR
remove_outliers <- function(df, column) {
  Q1 <- quantile(df[[column]], 0.25, na.rm = TRUE)
  Q3 <- quantile(df[[column]], 0.75, na.rm = TRUE)
  IQR <- Q3 - Q1
  lower_bound <- Q1 - 1.5 * IQR
  upper_bound <- Q3 + 1.5 * IQR
  df <- df[df[[column]] >= lower_bound & df[[column]] <= upper_bound, ]
  return(df)
}

# Apply outlier removal to both SD_CNA and SD_Exp
data <- remove_outliers(data, "SD_CNA")
data <- remove_outliers(data, "SD_Exp")

# 1. Scatter Plot: CNA Standard Deviation vs. Expression Standard Deviation
scatter_plot <- ggplot(data, aes(x = SD_CNA, y = SD_Exp)) +
  geom_point(alpha = 0.5, color = "black") +
  theme_minimal() +
  labs(title = "Variance Distribution: CNA vs. Expression",
       x = "CNA Standard Deviation",
       y = "Expression Standard Deviation")

# Save the scatter plot
ggsave(paste0(output_dir, "variance_distribution_refined.png"), plot = scatter_plot, width = 8, height = 6)

# 2. Categorize Genes Based on Variability
data <- data %>%
  mutate(CNA_Category = case_when(
    SD_CNA < quantile(SD_CNA, 0.33) ~ "Low",
    SD_CNA >= quantile(SD_CNA, 0.33) & SD_CNA < quantile(SD_CNA, 0.66) ~ "Moderate",
    SD_CNA >= quantile(SD_CNA, 0.66) ~ "High"
  )) %>%
  mutate(Exp_Category = case_when(
    SD_Exp < quantile(SD_Exp, 0.33) ~ "Low",
    SD_Exp >= quantile(SD_Exp, 0.33) & SD_Exp < quantile(SD_Exp, 0.66) ~ "Moderate",
    SD_Exp >= quantile(SD_Exp, 0.66) ~ "High"
  ))

# Save categorized data
write.csv(data, paste0(output_dir, "gene_variability_categories.csv"), row.names = FALSE)

# 3. Gene Interaction Network (Optimized for Speed)
# Select **top 500 genes** based on variance
top_variable_genes <- data %>%
  arrange(desc(SD_CNA + SD_Exp)) %>%
  head(500) %>%
  select(HGNC_Symbol, SD_CNA, SD_Exp)

# Generate random edges for adjacency matrix (simplified)
set.seed(42)
adj_matrix <- matrix(sample(0:1, 500^2, replace=TRUE, prob=c(0.9, 0.1)),
                     nrow = 500, ncol = 500)
colnames(adj_matrix) <- top_variable_genes$HGNC_Symbol
rownames(adj_matrix) <- top_variable_genes$HGNC_Symbol

# Convert to graph object
graph <- graph_from_adjacency_matrix(adj_matrix, mode = "undirected", diag = FALSE)
graph <- simplify(graph)  # Remove self-loops and multiple edges

# Filter nodes with low connections
graph <- delete.vertices(graph, V(graph)[degree(graph) < 5])  # Remove low-degree nodes

# Plot refined network (simplified)
png(paste0(output_dir, "gene_network_refined.png"), width = 800, height = 600)
plot(graph, vertex.size = 5, vertex.label.cex = 0.7, vertex.color = "lightblue",
     edge.color = "gray", main = "Refined Gene Interaction Network")
dev.off()

# 4. Clustering (Optimized for Speed)
# Convert to matrix with **only top 500 genes**
patient_matrix <- as.matrix(top_variable_genes[, c("SD_CNA", "SD_Exp")])
rownames(patient_matrix) <- top_variable_genes$HGNC_Symbol

# Hierarchical clustering
hc <- hclust(dist(patient_matrix))

# Save the clustering dendrogram
png(paste0(output_dir, "patient_clustering_refined.png"), width = 800, height = 600)
plot(hc, main = "Hierarchical Clustering of Highly Variable Genes", sub = "", xlab = "Genes", cex = 0.7)
dev.off()

# Save clustering data
write.csv(cutree(hc, k = 3), paste0(output_dir, "gene_clusters.csv"))


Loading required package: ggplot2

Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:igraph’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr


Attaching package: ‘tidyr’


The following object is masked from ‘package:igraph’:

    crossing


Loading required package: pheatmap

Warning message:
“The `adjmatrix` argument of `graph_from_adjacency_matrix()` must be symmetric
with mode = "undirected" as of igraph 1.6.0.
ℹ Use mode = "max" to achieve the original behavior.”
Warning message:
“`delete.vertices()` was deprecated in igraph 

pdf 
  2

pdf 
  2

In [1]:
# Install required packages if not already installed
list.of.packages <- c("ggplot2", "igraph", "ggraph", "tidyverse", "data.table", "factoextra", "dbscan")

new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)

# Load libraries
library(ggplot2)
library(igraph)
library(ggraph)
library(tidyverse)
library(data.table)
library(factoextra)
library(dbscan)

# Set working directory in Colab
setwd("/content/sample_data/Analysis_Results/")

# Ensure output folder exists
if (!dir.exists("Refined_Results")) {
  dir.create("Refined_Results")
}

# Load variance statistics (READ IN CHUNKS to avoid memory overload)
variance_file <- "/content/sample_data/Analysis_Results/variance_statistics.csv"
variance_data <- fread(variance_file, select = c("HGNC_Symbol", "Mean_CNA", "SD_CNA", "Mean_Exp", "SD_Exp"))

# Remove missing values
variance_data <- na.omit(variance_data)

### **🔹 Optimized Gene Network Analysis (Memory Efficient)**
set.seed(123)  # Ensure reproducibility
threshold <- 0.85  # High threshold to reduce memory load

# Compute correlations in pairs (Avoids full matrix explosion)
cor_results <- data.table()
for (i in 1:(nrow(variance_data) - 1)) {
  for (j in (i + 1):nrow(variance_data)) {
    gene1 <- variance_data$HGNC_Symbol[i]
    gene2 <- variance_data$HGNC_Symbol[j]

    cor_val <- cor(c(variance_data$Mean_CNA[i], variance_data$SD_CNA[i], variance_data$Mean_Exp[i], variance_data$SD_Exp[i]),
                   c(variance_data$Mean_CNA[j], variance_data$SD_CNA[j], variance_data$Mean_Exp[j], variance_data$SD_Exp[j]),
                   use = "pairwise.complete.obs")

    if (!is.na(cor_val) && abs(cor_val) >= threshold) {
      cor_results <- rbind(cor_results, data.table(Gene1 = gene1, Gene2 = gene2, Correlation = cor_val))
    }
  }
}

# Reduce memory usage by selecting top 200 strongest connections
cor_results <- cor_results[order(-abs(Correlation)), ][1:200, ]

# Convert to graph
gene_network <- graph_from_data_frame(d = cor_results, directed = FALSE)

# Refined network visualization
p <- ggraph(gene_network, layout = "kk") +
  geom_edge_link(aes(edge_alpha = abs(Correlation)), color = "gray50") +
  geom_node_point(size = 2, color = "blue") +
  geom_node_text(aes(label = name), repel = TRUE, size = 2.5) +
  theme_void() +
  ggtitle("Memory-Optimized Gene Network")

# Save network plot
ggsave("/content/sample_data/Analysis_Results/Refined_Results/gene_network_refined.png", p, width = 8, height = 6, dpi = 300)

### **🔹 Optimized Patient Clustering (Lower Memory Usage)**
# Scale data
scaled_data <- scale(variance_data[, .(Mean_CNA, SD_CNA, Mean_Exp, SD_Exp)])

# Use k-means clustering (instead of hierarchical)
set.seed(123)
kmeans_result <- kmeans(scaled_data, centers = 4, nstart = 10)

# Plot clustering
p2 <- fviz_cluster(kmeans_result, data = scaled_data, ellipse.type = "convex") +
  ggtitle("Optimized Patient Clustering")

# Save clustering
ggsave("/content/sample_data/Analysis_Results/Refined_Results/patient_clustering_refined.png", p2, width = 8, height = 6, dpi = 300)

### **✅ Final Outputs:**
# 1️⃣ **Optimized Gene Network** → `gene_network_refined.png`
# 2️⃣ **Optimized Patient Clustering** → `patient_clustering_refined.png`


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
✔ purrr     1.0.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::%--%()      masks igraph::%--%()
✖ dplyr::as_data_frame() masks tibble::as_data_frame(), igraph::as_data_frame()
✖ purrr::compose()       masks igraph::compose()
✖ tidyr::crossing()      masks igraph::crossing()
✖ dplyr::filter()        masks stats::filter()
✖ dplyr::lag()           masks stats::lag()
✖ purrr::simplify()      masks igraph::simplify()
ℹ Use the conflicted package (<http://conflicted.r-

In [2]:
# Install & load required libraries
list.of.packages <- c("ggplot2", "igraph", "ggraph", "tidyverse", "data.table", "factoextra", "dbscan")

new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)

# Load libraries
library(ggplot2)
library(igraph)
library(ggraph)
library(tidyverse)
library(data.table)
library(factoextra)
library(dbscan)

# Set working directory in Colab
setwd("/content/sample_data/Analysis_Results/")

# Ensure output folder exists
output_dir <- "Refined_Results"
if (!dir.exists(output_dir)) {
  dir.create(output_dir)
}

# Load variance statistics (Memory Efficient Read)
variance_file <- "/content/sample_data/Analysis_Results/variance_statistics.csv"
variance_data <- fread(variance_file, select = c("HGNC_Symbol", "Mean_CNA", "SD_CNA", "Mean_Exp", "SD_Exp"))

# Remove missing values
variance_data <- na.omit(variance_data)

# Normalize Data (Avoids large scales impacting correlation)
normalized_data <- variance_data %>%
  mutate(across(c(Mean_CNA, SD_CNA, Mean_Exp, SD_Exp), scale))

### **🔹 Fast Pairwise Correlation Computation (Vectorized)**
# Compute correlation matrix
cor_matrix <- cor(normalized_data[ , -1, with=FALSE], use = "pairwise.complete.obs")

# Convert to long format
cor_df <- as.data.frame(as.table(cor_matrix))
colnames(cor_df) <- c("Gene1", "Gene2", "Correlation")

# Filter only upper triangle to remove duplicates
cor_df <- cor_df[as.character(cor_df$Gene1) != as.character(cor_df$Gene2), ]

# Keep only strong correlations (above 0.85 absolute value)
cor_df <- cor_df[abs(cor_df$Correlation) >= 0.85, ]

# Reduce size by keeping top 300 strongest correlations
cor_df <- cor_df[order(-abs(cor_df$Correlation)), ][1:300, ]

# Convert into graph structure
gene_network <- graph_from_data_frame(cor_df, directed = FALSE)

### **🔹 Refined Gene Network Visualization**
p <- ggraph(gene_network, layout = "fr") +
  geom_edge_link(aes(edge_alpha = abs(Correlation)), edge_width = 0.5, color = "gray50") +
  geom_node_point(size = 3, color = "blue") +
  geom_node_text(aes(label = name), repel = TRUE, size = 2.5) +
  theme_void() +
  ggtitle("Refined Gene Network (High Quality)")

# Save the network plot
ggsave(paste0(output_dir, "/gene_network_refined.png"), p, width = 10, height = 8, dpi = 300)

### **🔹 Optimized Patient Clustering**
# Scale data for clustering
scaled_data <- scale(variance_data[, .(Mean_CNA, SD_CNA, Mean_Exp, SD_Exp)])

# Perform k-means clustering (Fast & Memory Efficient)
set.seed(123)
kmeans_result <- kmeans(scaled_data, centers = 4, nstart = 10)

# Visualize clustering
p2 <- fviz_cluster(kmeans_result, data = scaled_data, ellipse.type = "convex") +
  ggtitle("Optimized Patient Clustering")

# Save clustering plot
ggsave(paste0(output_dir, "/patient_clustering_refined.png"), p2, width = 8, height = 6, dpi = 300)

### **✅ Final Outputs:**
# 1️⃣ **High-Quality Gene Network** → `gene_network_refined.png`
# 2️⃣ **Optimized Patient Clustering** → `patient_clustering_refined.png`


Warning message:
“In `d`, `NA` elements were replaced with string "NA".”
Warning message in `[<-.factor`(`*tmp*`, thisvar, value = "NA"):
“invalid factor level, NA generated”
Warning message in `[<-.factor`(`*tmp*`, thisvar, value = "NA"):
“invalid factor level, NA generated”
Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_text_repel()`).”


### Start to use R in Colab
- Go to this URL: https://colab.research.google.com/#create=true&language=r, or this short URL https://colab.to/r

- Confirm that you are in the R runtime by going to the “Runtime” settings, and select “Change runtime type”

### Check and print out the R version

In [ ]:
R.version.string

[1] "R version 4.0.3 (2020-10-10)"

### Check packages available

In [ ]:
print(installed.packages())

             Package        LibPath                         Version   
IRdisplay    "IRdisplay"    "/usr/local/lib/R/site-library" "1.0"     
IRkernel     "IRkernel"     "/usr/local/lib/R/site-library" "1.1.1"   
pbdZMQ       "pbdZMQ"       "/usr/local/lib/R/site-library" "0.3-5"   
repr         "repr"         "/usr/local/lib/R/site-library" "1.1.3"   
uuid         "uuid"         "/usr/local/lib/R/site-library" "0.1-4"   
askpass      "askpass"      "/usr/lib/R/site-library"       "1.1"     
assertthat   "assertthat"   "/usr/lib/R/site-library"       "0.2.1"   
backports    "backports"    "/usr/lib/R/site-library"       "1.2.1"   
base64enc    "base64enc"    "/usr/lib/R/site-library"       "0.1-3"   
BH           "BH"           "/usr/lib/R/site-library"       "1.75.0-0"
blob         "blob"         "/usr/lib/R/site-library"       "1.2.1"   
brew         "brew"         "/usr/lib/R/site-library"       "1.0-6"   
brio         "brio"         "/usr/lib/R/site-library"       "1.1.1"   
broom 

## Install Package esquisse
- Explore and Visualize Your Data Interactively with the ggplot2 package.
- A 'shiny' gadget.
- To create 'ggplot2' charts interactively with drag-and-drop to map your variables.
- You can quickly visualize your data accordingly to their type, export to 'PNG' or 'PowerPoint', and retrieve the code to reproduce the chart.




In [ ]:
install.packages("esquisse")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘sass’, ‘jquerylib’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘bslib’, ‘miniUI’, ‘shiny’, ‘shinyWidgets’




In [ ]:
library(esquisse)

In [ ]:
install.packages("shiny")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(shiny)

- Load data

- Visualize the Data